# k-익명성

아래의 코드는 아래 Nuclearstar 원코드를 번역 및 주석처리
- [Original Github Page](https://github.com/Nuclearstar/K-Anonymity)

k-익명성 모델중 아래의 Mondrian 알고리즘을 구현한 모델임
- [Mondrian - Multidimensional k-Anonymity](https://www.utdallas.edu/~muratk/courses/privacy08f_files/MultiDim.pdf)


## k-익명성 구현

- 데이터셋에서 klt 프라이버시 보호를 만족하는 최적의 데이터를 찾는 문제는 NP-hard 문제임
- 근사 알고리즘을 통해 개인정보를 보호하는 문제를 해결하고자함

해당 코드는 몬드리안 알고리즘을 구현 하였고 몬드리안 알고리즘은 모든 속성을 숫자형 or 범주형 변수로 변환 한 후에 각 속성에 대해 'span' 값을 계산하여 익명화 시도

### 데이터 파티션

알고리즘은 다음의 단계를 통하여서 k익명성을 만족하는 그룹으로 분할:

1. 최종적으로 분할할 집합을 초기화 $P_{finished} = \{\}$.
2. 분할 해야 할 전체 데이터 집합을 초기화 $P_{working} = \{\{1, 2,\dots ,N\}\}$.
4. 분할 되지 않은 데이터가 있을 때 각각의 분할 집합에 대하여 :
  * 분할할 열에 대해 내림차순으로 정렬
  * 정렬된 각 열에 대해서 아래의 과정을 반복
      * 해당 열의 중앙값을 기준으로 데이터를 분할
      * 분할된 결과가 k-익명성을 만족하는지 체크
      * 만족하면 분할된 결과를 저장하고 반복문 종료
  * 분할을 생성하지 못했을 경우, 원본 열을 $P_{finished}$에 반한
5. 최종 분할된 집합$P_{finished}$을 반환

### 데이터 익명화

데이터를 분할한 후에 민감속성에 대해서 일반화를 통해서 준식별자가 k-익명성을 만족하도록 함


In [ ]:
# Pandas 패키지를 사용하여 데이터 프레임 작업을 할 수 있음
# import (패키지이름) as (코드내에서 사용할 약어)
import pandas as pd

In [ ]:
# 데이터의 열 이름 지정 (데이터셋에 열이름이 없음)
names = (
    'age',
    'workclass', 
    'fnlwgt', # 중복되는 사용자를 해당 가중치를 이용하여 표현
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income',
)

# 범주형 데이터에 대해 미리 지정해줌
#set 함수는 R에서 사용한 as.factor함수와 유사한 역할을 해줌
categorical = set((
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'sex',
    'native-country',
    'race',
    'income',
))
#데이터 입력 pandas패키지를 pd라는 약어로 사용하기로 정의 하였기 때문에 pd.read_csv를 통해서 데이터 입력
#해당 코드와 같은 폴더 내에 데이터가 존재 하므로 데이터 이름만 입력("adult.all.txt")
#sep을 통해 데이터의 구분자를 지정 해당 데이터의 경우 쉼표로 데이터가 구분되기 때문에 ", "
#데이터의 첫 행에 열 이름이 있을 경우 header변수에 True 값을 주면 된다.
df = pd.read_csv("adult.all.txt", sep=", ", header=None, names=names, index_col=False, engine='python');# We load the data using Pandas

In [ ]:
#jupter 노트 북의 경우 변수명을 실행하면 print 하게 됨 (어떤 에디터의 경우에는 print 함수를 사용해줘야함)
#pandas에서 제공하는 df.head() 를 통해 데이터의 상단 부분만 출력
df.head()

In [ ]:
#categorical 리스트에 있는 변수명에 대하여 astype('category') = 범주형 자료 로 변환
#R 데이터 프레임과 동일하게 열 번호 혹은 df['열이름']을 통해서 데이터를 불러 올 수 있음
for name in categorical:
    df[name] = df[name].astype('category')

**분할된 데이터 프레임의 모든 열에 대해 span을 계산하는 함수 구현** 

span

숫자형 데이터 : 최대값과 최소값의 차이

범주형 데이터 : 서로 구별데는 범주의 개수

In [ ]:
#python의 경우 def 함수명 (입력 변수)를 통해서 함수를 정의 함

def get_spans(df, partition, scale=None):
    """
    입력 변수 설명
            df: span을 계산할 데이터 프레임
     partition: span을 계산할 파티션
         scale: 스케일 값이 주어질 경우 각 열을 해당 스케일의 범위로 나눔
    출력 변수 설명
       returns: 파티션에 있는 모든 열의 span을 출력
    """
    #spans 라는 해시 태이블 정의
    spans = {}
    #각 열에 대하여
    for column in df.columns:
        #범주형 자료일 경우
        if column in categorical:
            span = len(df[column][partition].unique())
        #숫자혀 자료일 경우
        else:
            span = df[column][partition].max()-df[column][partition].min()
        #sacle이 입력 되었을 경우
        if scale is not None:
            span = span/scale[column]
        spans[column] = span
    return spans

In [ ]:
#df에 대하여 span 구하기
full_spans = get_spans(df, df.index)
full_spans

**데이터의 중앙 값을 기준으로 데이터를 분할하는 `split` 함수를 구현**

In [ ]:
def split(df, partition, column):
    """
    입력 변수 설명
            df: split 할 데이터 프레임
     partition: split 할 파티션
        column: split 할 열 지정
    출력 변수 설명
       returns: split된 결과 값
    """
    #해당 열의 해당 파티션 지정
    dfp = df[column][partition]
    #범주형 자료일 경우
    if column in categorical:
        #범주형 자료의 범주 추출 dfp.unique()
        values = dfp.unique()
        #범주를 반으로 나눔
        lv = set(values[:len(values)//2])
        rv = set(values[len(values)//2:])
        return dfp.index[dfp.isin(lv)], dfp.index[dfp.isin(rv)]
    #숫자형 자료일 경우
    else:
        #중앙 값
        median = dfp.median()
        dfl = dfp.index[dfp < median]
        dfr = dfp.index[dfp >= median]
        return (dfl, dfr)

위의 보조 함수를 사용하여 분할 알고리즘을 구현

**k-익명성을 만족시키는 데이터 분할(일반화) 알고리즘 구현**

In [ ]:
#k-익명성을 만족하는지 체크하는 함수
def is_k_anonymous(df, partition, sensitive_column, k=3):
    """
    입력 변수 설명
                   df: 파티션의 k익명성을 확인할 데이터 프레임
            partition: 데이터 프레임 내의 파티션
     sensitive_column: 민감 속성
                    k: k
    출력 변수 설명
    :returns         : k익명성을 만족하면 True 출력 만족하지 못하면 False를 출력
    """
    if len(partition) < k:
        return False
    return True

#데이터 분할 함수
def partition_dataset(df, feature_columns, sensitive_column, scale, is_valid):
    """
    입력 변수 설명
                   df: 파티션 할 데이터 프레임
      feature_columns: 데이터를 분할(일반화) 하는데 사용되는 열 이름 목록(준식별자)
     sensitive_column: 민감 속성 이름
                scale: span 함수의 scale
             is_valid: 데이터가 유효한 경우 True값을 출력
    출력 변수 설명
    :returns         : 파티션이 완료된 유효한 데이터 프레임
    """
    
    finished_partitions = []
    partitions = [df.index]
    while partitions:
        partition = partitions.pop(0)
        #각 열에 대해 span
        spans = get_spans(df[feature_columns], partition, scale)
        #정렬 후 split 함수를 적용하여 데이터 분할
        for column, span in sorted(spans.items(), key=lambda x:-x[1]):
            lp, rp = split(df, partition, column)
            if not is_valid(df, lp, sensitive_column) or not is_valid(df, rp, sensitive_column):
                continue
            partitions.extend((lp, rp))
            break
        else:
            finished_partitions.append(partition)
    return finished_partitions


전체 데이터를 진행 할 경우 알고리즘 수행시간이 오래 걸리기 때문에 간단한 예를 위해서 두개의 준 식별자와 하나의 민감정보만 선탁해여 결과를 확인 후 시각화

In [ ]:
# we apply our partitioning method to two columns of our dataset, using "income" as the sensitive attribute
feature_columns = ['age', 'education-num']
sensitive_column = 'income'
finished_partitions = partition_dataset(df, feature_columns, sensitive_column, full_spans, is_k_anonymous)

In [ ]:
# we get the number of partitions that were created
len(finished_partitions)

생성된 파티션을 시각화, 핕션의 사각형 경계를 얻는 함수를 사용

In [ ]:
 #matplotlib함수를 사용 하여 시각화 패키지내의 모듈만 import 하여 사용 할 수 있음
import matplotlib.pylab as pl
import matplotlib.patches as patches

In [ ]:
def build_indexes(df):
    indexes = {}
    for column in categorical:
        values = sorted(df[column].unique())
        indexes[column] = { x : y for x, y in zip(values, range(len(values)))}
    return indexes

def get_coords(df, column, partition, indexes, offset=0.1):
    if column in categorical:
        sv = df[column][partition].sort_values()
        l, r = indexes[column][sv[sv.index[0]]], indexes[column][sv[sv.index[-1]]]+1.0
    else:
        sv = df[column][partition].sort_values()
        next_value = sv[sv.index[-1]]
        larger_values = df[df[column] > next_value][column]
        if len(larger_values) > 0:
            next_value = larger_values.min()
        l = sv[sv.index[0]]
        r = next_value
    # we add some offset to make the partitions more easily visible
    l -= offset
    r += offset
    return l, r

def get_partition_rects(df, partitions, column_x, column_y, indexes, offsets=[0.1, 0.1]):
    rects = []
    for partition in partitions:
        xl, xr = get_coords(df, column_x, partition, indexes, offset=offsets[0])
        yl, yr = get_coords(df, column_y, partition, indexes, offset=offsets[1])
        rects.append(((xl, yl),(xr, yr)))
    return rects

def get_bounds(df, column, indexes, offset=1.0):
    if column in categorical:
        return 0-offset, len(indexes[column])+offset
    return df[column].min()-offset, df[column].max()+offset

In [ ]:
# we calculate the bounding rects of all partitions that we created
indexes = build_indexes(df)
column_x, column_y = feature_columns[:2]
rects = get_partition_rects(df, finished_partitions, column_x, column_y, indexes, offsets=[0.0, 0.0])

In [ ]:
# let's see how our rects look like
rects[:10]

In [ ]:
# we plot the rects
def plot_rects(df, ax, rects, column_x, column_y, edgecolor='black', facecolor='none'):
    for (xl, yl),(xr, yr) in rects:
        ax.add_patch(patches.Rectangle((xl,yl),xr-xl,yr-yl,linewidth=1,edgecolor=edgecolor,facecolor=facecolor, alpha=0.5))
    ax.set_xlim(*get_bounds(df, column_x, indexes))
    ax.set_ylim(*get_bounds(df, column_y, indexes))
    ax.set_xlabel(column_x)
    ax.set_ylabel(column_y)

In [ ]:
pl.figure(figsize=(20,20))
ax = pl.subplot(111)
plot_rects(df, ax, rects, column_x, column_y, facecolor='r')
pl.scatter(df[column_x], df[column_y])
pl.show()

# k-익명성을 만족하는 데이터 셋 생성

민감정보와 일반화된 준식별자만을 포함하는 새로운 k-익명성을 만족하는 데이터 셋을 생성 하려고 함

In [ ]:
#범주형 자료의 경우 ','문자열로 구분하여 데이터를 join하여 반환
def agg_categorical_column(series):
    return [','.join(set(series))]
#숫자 변수의 경우 평균값을 반환
def agg_numerical_column(series):
    return [series.mean()]

In [ ]:
#비식별화 데이터셋 만들기
def build_anonymized_dataset(df, partitions, feature_columns, sensitive_column, max_partitions=None):
    #해쉬 테이블 사용
    aggregations = {}
    #각 변수에 대해
    for column in feature_columns:
        #범주형 자료의 경우 agg_categorical_column함수를 사용
        if column in categorical:
            aggregations[column] = agg_categorical_column
        #숫자 변수의 경우 agg_numerical_column사용
        else:
            aggregations[column] = agg_numerical_column
    rows = []
    # 반복문 사용시 enumerate함수를 통해 loop가 실행 되었는지 확인 할 수 있음
    for i, partition in enumerate(partitions):
        #100으로 나눠서 나머지 1인 경우 몇번째 파티션 중인지 출력
        if i % 100 == 1:
            print("Finished {} partitions...".format(i))
        # 최대 반복수를 지정해주었을 경우 최대 반복수를 만족하면 알고리즘 종료
        if max_partitions is not None and i > max_partitions:
            break
        grouped_columns = df.loc[partition].agg(aggregations, squeeze=False)
        #민감열의 그룹별로 개수 count        
        sensitive_counts = df.loc[partition].groupby(sensitive_column).agg({sensitive_column : 'count'})
        values = grouped_columns.iloc[0].to_dict()
        
        for sensitive_value, count in sensitive_counts[sensitive_column].items():
            if count == 0:
                continue
            values.update({
                sensitive_column : sensitive_value,
                'count' : count,

            })
            rows.append(values.copy())
    return pd.DataFrame(rows)

In [ ]:
dfn = build_anonymized_dataset(df, finished_partitions, feature_columns, sensitive_column)

In [ ]:
# 민감 속성 기준으로 데이터 정렬
dfn.sort_values(feature_columns+[sensitive_column])

# l-다양성 구현 (the naive way)

k-익명성의 단점을 보완하기 위해 l-다양성 구현

* 데이터의 유효성을 검증하는 `is_valid` 값을 수정하여 l-다양성을 만족하도록 함
* `split` 함수를 수정하여 다양한 split을 생성 (가능 하다면)

**민감 정보 내의 파티션에 적어도 l개의 다른 민감한 속성 값이 존재하면 `True`, l-다양성을 만족하지 않으면 `False`를 반환하는 함수 작성**

In [ ]:
#해당 파티션내의 데이터의 다양성을 계산하는 함수
def diversity(df, partition, column):
    return len(df[column][partition].unique())

#데이터의 다양성이 l개 이상인지 체크하는 함수
def is_l_diverse(df, partition, sensitive_column, l=2):
    """
    입력 변 수 설명
    :param               df: The dataframe for which to check l-diversity
    :param        partition: The partition of the dataframe on which to check l-diversity
    :param sensitive_column: The name of the sensitive column
    :param                l: The minimum required diversity of sensitive attribute values in the partition
    출력 변수 설명
                           : True or False
    """
    
    return diversity(df, partition, sensitive_column) >= l

In [ ]:
#기존 분할 함수에 l-다양성 적용
finished_l_diverse_partitions = partition_dataset(df, feature_columns, sensitive_column, full_spans, lambda *args: is_k_anonymous(*args) and is_l_diverse(*args))

In [ ]:
len(finished_l_diverse_partitions)

In [ ]:
column_x, column_y = feature_columns[:2]
l_diverse_rects = get_partition_rects(df, finished_l_diverse_partitions, column_x, column_y, indexes, offsets=[0.0, 0.0])

In [ ]:
pl.figure(figsize=(20,20))
ax = pl.subplot(111)
plot_rects(df, ax, l_diverse_rects, column_x, column_y, edgecolor='b', facecolor='b')
plot_rects(df, ax, rects, column_x, column_y, facecolor='r')
pl.scatter(df[column_x], df[column_y])
pl.show()

In [ ]:
# ㅣ다양성을 만족시키는 익명화 셋 생성
dfl = build_anonymized_dataset(df, finished_l_diverse_partitions, feature_columns, sensitive_column)

In [ ]:
# 데이터셋 정렬
dfl.sort_values([column_x, column_y, sensitive_column])

# t-근접성 구현

l-다양성 데이터의 경우 각각 민감 속성이 다양하게 분포한 반면에 준식별자의 경우 다양성이 떨어지게 분포하게 됨, 실제 데이터셋과 익명화된 데이터셋의 분포 차이에서 오는 개인정보 유출을 막기 위해 t-근접성 적용

**the Kolmogorov-Smirnov 거리를 통해 민감 속성과 전체 데이터 사이의 분포 거리를 계산 (the Kolmogorov-Smirnov 거리는 두 분포의 누적확률 분포의 거리를 나타내는 함수로 Kolmogorov-Smirnov test를 통해서 두 분포가 같은 분포인지 아닌지 테스트 할 수 있음
**

In [ ]:
# 민감 속성에 대하여 전체 분포를 생성 할 수 있음
global_freqs = {}
total_count = float(len(df))
group_counts = df.groupby(sensitive_column)[sensitive_column].agg('count')
for value, count in group_counts.to_dict().items():
    p = count/total_count
    global_freqs[value] = p

In [ ]:
global_freqs

In [ ]:
def t_closeness(df, partition, column, global_freqs):
    total_count = float(len(partition))
    d_max = None
    group_counts = df.loc[partition].groupby(column)[column].agg('count')
    for value, count in group_counts.to_dict().items():
        p = count/total_count
        d = abs(p-global_freqs[value])
        if d_max is None or d > d_max:
            d_max = d
    return d_max


def is_t_close(df, partition, sensitive_column, global_freqs, p=0.2):
    """
    입력 변수 설명
    :param               df: The dataframe for which to check l-diversity
    :param        partition: The partition of the dataframe on which to check l-diversity
    :param sensitive_column: The name of the sensitive column
    :param     global_freqs: The global frequencies of the sensitive attribute values
    :param                p: The maximum allowed Kolmogorov-Smirnov distance
    """
    if not sensitive_column in categorical:
        raise ValueError("this method only works for categorical values")
    return t_closeness(df, partition, sensitive_column, global_freqs) <= p

In [ ]:
# 비식별화에 적용
finished_t_close_partitions = partition_dataset(df, feature_columns, sensitive_column, full_spans, lambda *args: is_k_anonymous(*args) and is_t_close(*args, global_freqs))

In [ ]:
len(finished_t_close_partitions)

In [ ]:
dft = build_anonymized_dataset(df, finished_t_close_partitions, feature_columns, sensitive_column)

In [ ]:
# Let's see how t-closeness fares
dft.sort_values([column_x, column_y, sensitive_column])

In [ ]:
column_x, column_y = feature_columns[:2]
t_close_rects = get_partition_rects(df, finished_t_close_partitions, column_x, column_y, indexes, offsets=[0.0, 0.0])

In [ ]:
pl.figure(figsize=(20,20))
ax = pl.subplot(111)
plot_rects(df, ax, t_close_rects, column_x, column_y, edgecolor='b', facecolor='b')
pl.scatter(df[column_x], df[column_y])
pl.show()